In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas

Welcome to JupyROOT 6.16/00


In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


In [78]:
def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData = Timepix.Decode(nHitsA,encodedA)
        TimepixBData = Timepix.Decode(nHitsB,encodedB)
        
        def sortSecond(val): 
            return val[2]  
        
        TimepixAData.sort(key=sortSecond) 


aq.runFunction(plotTimepix,0,50000)        

TypeError: 'key' is an invalid keyword argument for this function

In [72]:

# Python program to demonstrate sorting by user's 
# choice 
  
# function to return the second element of the 
# two elements passed as the paramater 
def sortSecond(val): 
    return val[2]  
  
# list1 to demonstrate the use of sorting  
# using using second key  
list1 = [(1, 2, 3), (3, 3, 1), (1, 1, 2)] 
  
# sorts the array in ascending according to  
# second element 
list1.sort(key = sortSecond)  
print(list1) 
  
# sorts the array in descending according to 
# second element 
list1.sort(key = sortSecond, reverse = True) 
print(list1) 


[(3, 3, 1), (1, 1, 2), (1, 2, 3)]
[(1, 2, 3), (1, 1, 2), (3, 3, 1)]


In [76]:
ROOT.enableJSVis()

tdposA = Hist2D(256,0,256,256,0,256)

c4 = ROOT.TCanvas("my4","Clusters", 1000,1000)

def sortThird(val):
    return val[2]


def plotTimepix():
    cl = [None]*100          # a cluster can hold up to 100 hits
    cn = 0                   # counts number of hits in current cluster
    blen = 0                 # the length of the current tp3 buffer (?)
    b = 0                    # the running (current?) buffer position
    nc = 0                   # count the number of clusters found

    if(aq.epicsEvent==1): # each time it goes through this is a buffer?
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        TimepixAData = Timepix.Decode(nHitsA,encodedA) # nHitsA == len(TimepixAData)
        TimepixAData.sort(key = sortThird)
        
        blen = nHitsA

        while(b<blen):        # for every buffer position in the buffer?
            cn = 0            # zero the cluster counter for the current buffer position
            cl[cn] = b        # assume the current position has the first hit in the cluster and save it#
            cn+=1             # increment the counter, since we've counted a hit
            # this next bit was a for loop in Ken's code
            c=b+1             # start at position c that is one above the current buffer position
            while((c<b+99) and (c<blen)): # check all positions up to 99 away, as long as we're still in the buffer
                if(abs(TimepixAData[c]['ToA']- TimepixAData[b]['ToA'] < 100)):   # if an event at c is within 100ns of that at the current buffer position
                    cl[cn] = c                             # save this position to the cluster
                    cn+=1                                  # increment the counter, since we've counted a hit
                else:
                    process_cluster(TimepixAData)               # analyze the cluster we found
                    b+=cn                           # move on beyond this cluster
                    cn = 0                          # reset the counter for hits in the cluster
                    cl = [None]*100                 # reset the array of cluster hits
                    nc+=1                           # increment the number of clusters found

                    if(c<b+99 and c<blen):          # if we're still in the loop
                        cl[cn] = b                  # assume current position is the first in the new cluster
                        cn+=1                       # increment the counter, since we've counted a hit

                c+=1 # move to the next position and go through this loop again
            b+=1
    

    tdposA.Draw("colz")
    c4.Draw()
        
        
        
aq.runFunction(plotTimepix,0,50000)

TypeError: 'key' is an invalid keyword argument for this function

In [60]:
def process_cluster(TimepixAData):
    start_index = cl[0]                  # start index of the first hit in the cluster
    print(start_index)
    min_time = TimepixAData['ToA'][start_index]     # time of the first hit in the cluster
    dtime = 0.0                          # time difference?
    meanx = 0.0
    meany = 0.0
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):             # go through each hit in the cluster
        if(TimepixAData['ToA'][n] < min_time):      # if we find a time smaller than the current min time
            min_time = TimepixAData['ToA'][n]      # reassign the smaller time to be the min time
        meanx+=TimepixAData['x'][n]               # add up all the x positions
        meany+=TimepixAData['y'][n]               # add up all the y positions
        n+=1                             # move to the next hit in the cluster

    meanx/=cn                        # calculate the mean x position
    meany/=cn                        # calculate the mean y position
    
    tdposA.Fill(meanx,meany)         # plot mean y vs. mean x
    multA.Fill(cn)                   # plot the counts in the cluster
    
    # this next bit was a for loop in Ken's code
    n = start_index
    while(n<start_index+cn):               # for each hit in the cluster
        dtime = TimepixAData['ToA'][n] - min_time     # find time difference between a hit in the cluster and the min time of the cluster
        dthistA.Fill(dtime)                # put the time difference in the histogram
        dtime_toaA.Fill(dtime,TimepixAData[n][1])
        n+=1                               # move to the next hit in the cluster